In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math
import fitClus2D as fc
from scipy.stats import sigmaclip
from astropy.stats import sigma_clip
from scipy.optimize import fmin_tnc

from sklearn.model_selection import LeaveOneOut
from sklearn.neighbors import KNeighborsClassifier, RadiusNeighborsClassifier
from sklearn. metrics import accuracy_score, mean_squared_error
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.neural_network import MLPClassifier
from sklearn import preprocessing
import pandas as pd

zclus = {'a2744' : 0.308,
         'm0416' : 0.396}
pgals = {'a2744' : [1.689791e-01, 1.965115e+00, 2.0],
        'm0416' : [3.737113e-01, 1.322081e+00, 2.0]}

In [2]:
cluster = 'm0416'
catalog = "Files/" + cluster + "_master_cleaned.csv"

dzcut=0.02; sigclip=3; sigcut=3; radialcut=120; maglim=23.5; colorcut=1.0
master = pd.read_csv(catalog)
zdat = master['master_z']
valid = master['master_valid']
memberindx = np.where((zdat>zclus[cluster]-dzcut)&(zdat<zclus[cluster]+dzcut)&(valid==1))[0]
z1 = zdat[memberindx]
z2 = sigma_clip(z1,sigma=sigclip)
print(np.mean(z2),np.std(z2))

dz = sigcut*np.std(z2)
print(dz)
ddat = master['master_d']
zdat = master['master_z']
mag814 = master['master_mag814']
mag606 = master['master_mag606']
valid = master['master_valid']
master_cut = master.iloc[np.where((ddat<radialcut)&(~np.isnan(mag606))&(~np.isnan(mag814))&(mag814<=maglim)&(valid == 1))]

zdat   = master_cut['master_z']
nmemindx = np.where((zdat<zclus[cluster]-dz)|(zdat>zclus[cluster]+dz))[0]
memindx = np.where((zdat>zclus[cluster]-dz)&(zdat<zclus[cluster]+dz))[0]
unknownindx = np.where(np.isnan(zdat))[0]

print(len(memindx))
print(len(nmemindx))
print(len(unknownindx))
clusterarr = [master_cut.iloc[i] for i in memindx]
ncluster =[master_cut.iloc[i] for i in nmemindx]
unknown = [master_cut.iloc[i] for i in unknownindx]

0.3960252040816326 0.007215352640991353
0.02164605792297406
146
90
125


In [3]:
#Create feature and category arrays for knowns, and feature array for unknowns
XclustCM= [[row['master_mag814'], row['master_mag606'] - row['master_mag814']] for row in clusterarr]
XnclustCM = [[row['master_mag814'], row['master_mag606'] - row['master_mag814']] for row in ncluster]
Xcm = np.array(XclustCM + XnclustCM)

unknownXcm = [[row['master_mag814'], row['master_mag606'] - row['master_mag814']] for row in unknown]
unknownXcm = np.array(unknownXcm)

clustnum = len(clusterarr)
nclustnum = len(ncluster)

print(clustnum)

y = np.zeros(clustnum + nclustnum)
y[:len(clusterarr)] = 1

print(y)

146
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [5]:
clustermag = [dat[0] for i, dat in enumerate(Xcm) if y[i] == 1]
clustercolor = [dat[1] for i, dat in enumerate(Xcm) if y[i] == 1]
print(len(clustercolor))

model = np.polyfit(clustermag, clustercolor, 1)
predict = np.poly1d(model)
res = clustercolor - predict(clustermag)
    
sigma = np.std(res)

inds = np.where(abs(unknownXcm[:,1] - predict(unknownXcm[:,0])) < sigma)[0]
photmembers = [unknown[i] for i in inds]

print(len(photmembers))

146
61


In [10]:
papercatalog = "Files/" + cluster + "paper.csv"
paper = pd.read_csv(papercatalog)
status = paper['Status']
paperinds = np.where((status == "member-phot") | (status == "member-spec"))[0]
papermem = [paper.iloc[i] for i in paperinds]

print(len(papermem))

207


In [18]:
pd.set_option("display.max_rows", None, "display.max_columns", None)
mygals = fc.galclass("m0416-def-sigma.txt")

mydat = mygals.dat

included = [False for i in range(len(mydat))]
used = [False for i in range(len(papermem))]

count = 0
for i, gal in enumerate(mydat):
    for j, gal2 in enumerate(papermem):
        if abs(gal - gal2.RA) < 0.00001 and abs(gal.master_Dec - gal2.Dec) < 0.00001:
            if abs(gal.master_mag606 - gal2.F606W_mag) > 0.02 or abs(gal.master_mag814 - gal2.F814W_mag) > 0.02:
                count += 1
                print(count)
            included[i] = True
            used[j] = True
            
print(included)
print(used)

61
[True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True]
[True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True]


In [14]:
specinds = np.where(status == "member-spec")[0]
paperspec = [paper.iloc[i] for i in specinds]

included = [False for i in range(len(clusterarr))]
used = [False for i in range(len(paperspec))]

print(len(specinds))


for i, gal in enumerate(clusterarr):
    for j, gal2 in enumerate(paperspec):
        if abs(gal.master_RA - gal2.RA) < 0.00001 and abs(gal.master_Dec - gal2.Dec) < 0.00001:
            included[i] = True
            used[j] = True
            
print(included)
print(used)

146
[True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True]
[True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True